<a href="https://colab.research.google.com/github/alibekk93/NLP_practice/blob/main/Micrograd_tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Andrej Karpathy's Micrograd tutorial from https://www.youtube.com/watch?v=VMj-3S1tku0 and https://github.com/karpathy/nn-zero-to-hero/tree/master/lectures/micrograd

https://karpathy.ai/zero-to-hero.html

# Libraries

In [2]:
from tqdm import tqdm
import math
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline